<a href="https://colab.research.google.com/github/eduardadelavle/projeto-InteligenciaArtificial/blob/master/agmochila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
import numpy as np
import pandas as pd
import random as rd
from random import randint
import matplotlib.pyplot as plt
import random
 #utilização do random para questões aleatórias

**Criação da mochila e dos itens**

In [127]:
itens = np.arange(1,11)

#randint nesse caso abaixo gera um vetor aleatório de 10 posições com valores de 1 até 15
peso = np.random.randint(1, 15, size = 10)    #10 pesos aleatorios entre 1 e 15
valor = np.random.randint(10, 150, size = 10) #10 valores entre 10 e 150
peso_maximo_mochila = 20    
print('Item No.   Peso   Valor')
#shape pega o tamanho do vetor 
for i in range(itens.shape[0]):  #shape retorna quantidade de elemento de um vetor
    print('{0}          {1}         {2}\n'.format(itens[i], peso[i], valor[i]))

Item No.   Peso   Valor
1          3         99

2          1         26

3          11         54

4          2         72

5          8         115

6          7         142

7          1         16

8          6         140

9          9         25

10          11         39



**Cria população inicial aleatóriamente e definição o número de gerações (iterações) que o processo de busca irá ocorrer**

In [120]:
tamanho_populacao = 10

#Esse comando abaixo serve para estabelecer um vetor onde temos 10 linhas com 10 colunas
pop_size = (tamanho_populacao, itens.shape[0])

print('Tamanho da População = {}'.format(pop_size))

populacao_inicial = np.random.randint(2, size = pop_size)
populacao_inicial = populacao_inicial.astype(int)
num_geracoes = 100
print('Populacao Inicial: \n{}'.format(populacao_inicial))

# 1 significa tem tem o item na mochila e 0 não

Tamanho da População = (10, 10)
Populacao Inicial: 
[[0 1 1 0 1 1 0 1 0 0]
 [0 0 1 1 1 0 1 0 1 1]
 [0 1 0 1 1 1 1 1 0 1]
 [1 1 1 0 1 0 0 0 1 1]
 [1 1 0 1 0 0 1 1 1 1]
 [0 1 1 0 0 1 1 1 0 0]
 [1 1 0 0 0 0 1 1 0 0]
 [0 0 1 0 1 0 1 1 1 0]
 [1 1 0 1 0 0 0 0 1 0]
 [0 1 0 1 1 1 0 1 1 0]]


**Calcula o fitness da população existente**

In [128]:
#Calcula Fitness
def calc_fitness(peso, valor, populacao, peso_maximo_mochila): #valor cabe na mochila
    fitness = np.empty(populacao.shape[0])
    for i in range(populacao.shape[0]):
        a1 = np.sum(populacao[i] * valor)
        a2 = np.sum(populacao[i] * peso) # verifica o peso e multiplica pela posição, depois soma 
        if a2 <= peso_maximo_mochila:  #verifica se o peso n ultrapassou o de 20
            fitness[i] = 1  # se for menor que 20 ele guarda
        else :
            fitness[i] = 0   # se for maior ele elimina
    return fitness.astype(int)   

**Executa o processo** - aplicação do genético


In [130]:
def optimize(pesos, valor, population, pop_size, num_geracoes, peso_maximo_mochila):
    parameters, fitness_history = [], []
    
    num_pais_selecao = int(pop_size[0]*0.6) #numero de pais que participam da seleção cruzamento, mutação
    #faz a evolução do processo durante o número de gerações
    for i in range(num_geracoes):
        #calcula o fitness da população atual
        fitness = calc_fitness(pesos, valor, population, peso_maximo_mochila)
        
        #elitismo - entra a partir da 2° geração, mantém os melhores individuios
        population=population[fitness.argsort()[-pop_size[0]:][::-1]]
        fitness=fitness[fitness.argsort()[-pop_size[0]:][::-1]]
        #fitness.argsort: ordena do menor para maior.
        
        fitness_history.append(fitness) #imprime gráfico

        #seleciona os pais
        pais = selection(fitness, num_pais_selecao, population)

        #realiza o cruzamento
        filhos = crossover(pais)

        #realiza a mutação
        mutants = mutation(filhos)

        #insere novos filhos na populacao
        if(mutants.size != 0.):
          population=np.append(population, mutants, axis = 0)
        
        #Mantém apenas vetores únicos, ignorando repetições
        population, index = np.unique(population, axis=0, return_index=True)

        
    print('Última Geração: \n{}\n'.format(population)) 
    fitness_last_gen = calc_fitness(pesos, valor, population, peso_maximo_mochila)  

    print('Fitness da última geração: \n{}\n'.format(fitness_last_gen))
    max_fitness = np.where(fitness_last_gen == np.max(fitness_last_gen))
    
    parameters.append(population[max_fitness[0][0],:])
    return parameters, fitness_history

**Operação de seleção (usando torneio)**

In [131]:
def selection(fitness2, num_pais, population):
    fitness = list(fitness2)
    parents = np.empty((num_pais, population.shape[1]))
    
    #taxa de comparação
    k=0.6
    for i in range(num_pais):

        #seleciona aleatoriamente 2 individuos da população
        x1=random.randint(0, population.shape[0]-1)
        x2=random.randint(0, population.shape[0]-1)

        #Verifica se já não foram utilizados, se foram utilizados tenho
        #que selecionar outro
        while (fitness[x1]==-999999):
          x1=random.randint(0, population.shape[0]-1)
        while (fitness[x2]==-999999):
          x2=random.randint(0, population.shape[0]-1)

        #Ajuste para o individuo de maior fitness sempre ser o x1
        if (fitness[x1]<fitness[x2]):
          aux=x1
          x1=x2
          x2=aux
      
        r=random.random() #valor aleatorio
        # seleciono fitness de maior valor no primeiro caso(x1) e pior valor no segundo(x2)
        if (r<k): 
          parents[i,:] = population[x1, :]
          fitness[x1] = -999999
        else:
          parents[i,:] = population[x2, :]
          fitness[x2] = -999999
    # retorno todos pais selecionados
    return parents
  #para esse caso a chance é de encontrar melhor fitness por porcentagem é maior(60%).

**Cruzamento**

In [132]:
def crossover(parents):
    #gerando um vetor para armazenar os filhos
    vetFilhos = []
    #determinando que no cruzamento vou pegar o meio de cada individuo
    crossover_point = int(parents.shape[1]/2)

    #estabelecendo uma taxa do cruzamento ocorrer
    crossover_rate = 0.8
    i=0
    while (i < parents.shape[0]):
        x = rd.random()
        if x < crossover_rate:
          parent1_index = i%parents.shape[0]
          parent2_index = (i+1)%parents.shape[0]
          
          #pega a metade de cada pai para fazer os filhos
          #f1 metade do Pai 1 e outra do Pai 2
          #f2 metade do Pai 1 e outra do Pai 2

          f1 = np.empty(parents.shape[1])
          f2 = np.empty(parents.shape[1])

          f1[0:crossover_point] = parents[parent1_index,0:crossover_point]
          f1[crossover_point:]  = parents[parent2_index,crossover_point:]
          
          f2[0:crossover_point] = parents[parent2_index,0:crossover_point]
          f2[crossover_point:]  = parents[parent1_index,crossover_point:]
          
          
         vetFilhos.append(f1)
          vetFilhos.append(f2)
        i=i+2
    return np.asarray(vetFilhos, dtype=np.int32)

**Mutação**

In [133]:
def mutation(vetFilhos):
    mutants = np.empty((vetFilhos.shape))
    mutation_rate = 0.3
    for i in range(mutants.shape[0]):
        random_value = rd.random()
        mutants[i,:] = vetFilhos[i,:]
        if random_value > mutation_rate:
            continue
        int_random_value = randint(0,vetFilhos.shape[1]-1)    
        if mutants[i,int_random_value] == 0 :
            mutants[i,int_random_value] = 1
        else :
            mutants[i,int_random_value] = 0
    return mutants

In [126]:
parameters, fitness_history = optimize(peso, valor, populacao_inicial, pop_size, num_geracoes, peso_maximo_mochila)
print('A solução que de saída é: \n{}'.format(parameters))
itens_selecionados = itens * parameters
print('\nOs item que irão maximizar a mochila são:')
total_peso=0
total_valor=0
for i in range(itens_selecionados.shape[1]):
  if itens_selecionados[0][i] != 0:
     print('{0}          {1}         {2}\n'.format(itens[i], peso[i], valor[i]))
     total_peso=total_peso+peso[i]
     total_valor=total_valor+valor[i]
print('      Peso: {0}        Valor {1}\n'.format(total_peso,total_valor))

Última Geração: 
[[0. 0. 0. 0. 1. 1. 1. 0. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]
 [0. 0. 1. 0. 1. 1. 0. 0. 1. 1.]
 [0. 0. 1. 0. 1. 1. 1. 0. 0. 1.]
 [0. 0. 1. 0. 1. 1. 1. 0. 1. 1.]
 [0. 0. 1. 0. 1. 1. 1. 1. 0. 1.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1. 1.]
 [1. 0. 0. 0. 1. 0. 1. 0. 1. 1.]
 [1. 0. 0. 0. 1. 1. 0. 0. 1. 1.]
 [1. 0. 0. 0. 1. 1. 1. 0. 0. 1.]
 [1. 0. 0. 0. 1. 1. 1. 1. 0. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 1.]]

Fitness da última geração: 
[459 476 442 453 405 506 422 427 480 491 443 460   0]

A solução que de saída é: 
[array([0., 0., 1., 0., 1., 1., 1., 0., 1., 1.])]

Os item que irão maximizar a mochila são:
3          7         47

5          5         140

6          1         64

7          1         53

9          5         101

10          1         101

      Peso: 20        Valor 506

